In [1]:
model_name = 'shakespeare-10-30.pl'
word2id_name = 'shakespeare-10-30-word2id.pl'
models_dir = 'models/'

In [4]:
from src.models import Skipgram_SM
import torch
import torch.nn.functional as F

In [6]:
word2id = torch.load(models_dir + word2id_name)
id2word = {i: word for word, i in word2id.items()}
model = Skipgram_SM(len(word2id), 10)
model.load_state_dict(torch.load(models_dir + model_name))

/tmp/ipykernel_44229/3154820269.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  word2id = torch.load(models_dir + word2id_name)
/tmp/ipykernel_44229/3154820269.py:4: Fut

<All keys matched successfully>

In [10]:
find_closest('body')

[(0.8826966285705566, 'pressing'),
 (0.8735028505325317, 'slit'),
 (0.8664693236351013, 'gory'),
 (0.8242490291595459, 'vouch'),
 (0.8182522654533386, 'whither'),
 (0.8079332113265991, 'shepherd'),
 (0.7905085682868958, 'continuance'),
 (0.7885518670082092, 'neglecting'),
 (0.787747323513031, 'wretchedness'),
 (0.7865263223648071, 'twinn'),
 (0.7863031625747681, 'cheap'),
 (0.7855322360992432, 'silver'),
 (0.7762913703918457, 'redeliver'),
 (0.7687159776687622, 'envy'),
 (0.7682114243507385, 'perjury'),
 (0.7574726939201355, 'easiest'),
 (0.757323145866394, 'temperately'),
 (0.7516161799430847, 'erpingham'),
 (0.7515144348144531, 'thracian'),
 (0.7512025833129883, 'serpigo')]

In [8]:
def find_closest(target, model=model, word2id=word2id, id2word=id2word, top_k=20):
    target_idx = word2id[target]
    target_embed = model.embed(torch.tensor([target_idx]))  # Shape (1, embedding_dim)
    
    # Get all embeddings at once
    all_embeds = model.embed.weight  # Shape (num_embeddings, embedding_dim)
    
    # Compute cosine similarities between target and all other embeddings
    similarities = F.cosine_similarity(target_embed, all_embeds, dim=1)  # Shape (num_embeddings,)
    
    # Set similarity of the target word to a very low value to exclude it from results
    similarities[target_idx] = -float('inf')
    
    # Get the top K most similar embeddings
    top_k_indices = torch.topk(similarities, top_k).indices  # Shape (top_k,)
    
    # Retrieve corresponding words and similarities
    closest_words = [(similarities[i].item(), id2word[int(i)]) for i in top_k_indices]
    
    return closest_words